# Exportación de librerias

In [759]:
#Importación de librerias
import sqlite3
import pandas as pd
import numpy as np
import openpyxl
from pathlib import Path
import re

pd.set_option('display.max_columns', None)

# Lectura de bases de datos

In [760]:
#Lectura de bases de datos
base_path = Path().resolve()

#BD Municipios
mun_path = base_path / "Bases de datos" / "Municipios.xlsx"
municipios = pd.read_excel(mun_path, engine="openpyxl")

#BD Prestadores 13/05/2025
prest_path = base_path / "Bases de datos" / "Prestadores.xlsx"
prestadores = pd.read_excel(prest_path, engine="openpyxl")

In [761]:
#BD Proyección poblacional 2020-2029
pob_path = base_path / "Bases de datos" / "Proy_Pob_2020-2029.xlsx"
pob = pd.read_excel(pob_path, engine="openpyxl")

In [762]:
#BD Poblacional por etnia
etn_path = base_path / "Bases de datos" / "Proy_Pob_EtnicoRacial.xlsx"
etn = pd.read_excel(etn_path, engine="openpyxl")

# Limpieza de datos

In [763]:
#Función de limpieza de texto
def limpiar_texto(texto):
    if pd.isna(texto):
        return texto
    texto = texto.strip()  # Quita espacios al inicio y final
    # Elimina caracteres que no sean letras, números, espacios, ni acentos (con regex unicode)
    texto = re.sub(r"[^\w\sáéíóúÁÉÍÓÚñÑüÜ]", "", texto, flags=re.UNICODE)
    texto = texto.upper()  # Mayúsculas para uniformidad
    return texto

## Limpieza de BD Prestadores

In [764]:
# Convertimos a mayúsculas para que coincidan con los datos en la vista prest_agr
prestadores['depa_nombre'] = prestadores['depa_nombre'].apply(limpiar_texto)
prestadores['muni_nombre'] = prestadores['muni_nombre'].apply(limpiar_texto)

## Limpieza de BD Municipios

In [765]:
# Aplicándolo a columnas
municipios['Departamento'] = municipios['Departamento'].apply(limpiar_texto)
municipios['Municipio'] = municipios['Municipio'].apply(limpiar_texto)

In [766]:
#Buscar datos vacios
total_filas = municipios.shape[0]
print(total_filas)
columnas = ['DP', 'Departamento', 'MPIO', 'Municipio', 'Superficie', 'PopTot', 'Rural', 'Region']
print(municipios[columnas].isna().sum())


1118
DP              0
Departamento    0
MPIO            0
Municipio       0
Superficie      1
PopTot          0
Rural           1
Region          0
dtype: int64


In [767]:
filas_con_vacios = municipios[municipios[columnas].isna().any(axis=1)]
print(filas_con_vacios)

      DP Departamento   MPIO   Municipio  Superficie  PopTot  Rural  \
1097  94      GUAINÍA  94663  MAPIRIPANA         NaN       0    NaN   

            Region  
1097  Región Llano  


In [768]:
#Debido a que la fila de municipio 94663 perteneciente a Guainía - Mapiripana esta vacia, se realiza una investigación y se encuentra que el municipio desde diciembre del 2019 paso a ser corregimiento del municipio Barrancominas. Por lo tanto se elimina esa linea. 
municipios = municipios.dropna(subset=columnas)
#Verificacion de la limpieza - se espera que este vacío
filas_con_vacios = municipios[municipios[columnas].isna().any(axis=1)]
print(filas_con_vacios)

Empty DataFrame
Columns: [DP, Departamento, MPIO, Municipio, Superficie, PopTot, Rural, Region]
Index: []


In [769]:
#Las columnas de se van renombrar conforme al archivo del ministerio para facilidad en el flujo futuro. 
municipios = municipios.rename(columns={
    'Departamento': 'depa_nombre',
    'Municipio': 'muni_nombre'
})
municipios.head(5)

,DP,depa_nombre,MPIO,muni_nombre,Superficie,PopTot,Rural,Region
0,5,ANTIOQUIA,5001,MEDELLÍN,350.666623,2634570,1.6,Región Eje Cafetero
1,5,ANTIOQUIA,5002,ABEJORRAL,497.566212,21622,56.7,Región Eje Cafetero
2,5,ANTIOQUIA,5004,ABRIAQUÍ,287.641603,2872,64.2,Región Eje Cafetero
3,5,ANTIOQUIA,5021,ALEJANDRÍA,119.467683,4989,40.7,Región Eje Cafetero
4,5,ANTIOQUIA,5030,AMAGÁ,89.245903,32628,47.4,Región Eje Cafetero


## Limpieza de BD Proyección Población

In [770]:
#Mayusculas para uniones con archivo Municipio después
pob['DPNOM'] = pob['DPNOM'].str.upper()
pob['DPMP'] = pob['DPMP'].str.replace(r'\s*\([^)]*\)', '', regex=True).str.strip().str.upper()
#Eliminamos municipio 94663 - Guainía - Mapiripana
pob = pob[pob['MPIO'] != 94663]
#Eliminamos población de antes del 2025
pob = pob[pob['AÑO'] >= 2025]
pob

,DP,DPNOM,MPIO,DPMP,AÑO,ÁREA GEOGRÁFICA,Hombres_0,Hombres_1,Hombres_2,Hombres_3,Hombres_4,Hombres_5,Hombres_6,Hombres_7,Hombres_8,Hombres_9,Hombres_10,Hombres_11,Hombres_12,Hombres_13,Hombres_14,Hombres_15,Hombres_16,Hombres_17,Hombres_18,Hombres_19,Hombres_20,Hombres_21,Hombres_22,Hombres_23,Hombres_24,Hombres_25,Hombres_26,Hombres_27,Hombres_28,Hombres_29,Hombres_30,Hombres_31,Hombres_32,Hombres_33,Hombres_34,Hombres_35,Hombres_36,Hombres_37,Hombres_38,Hombres_39,Hombres_40,Hombres_41,Hombres_42,Hombres_43,Hombres_44,Hombres_45,Hombres_46,Hombres_47,Hombres_48,Hombres_49,Hombres_50,Hombres_51,Hombres_52,Hombres_53,Hombres_54,Hombres_55,Hombres_56,Hombres_57,Hombres_58,Hombres_59,Hombres_60,Hombres_61,Hombres_62,Hombres_63,Hombres_64,Hombres_65,Hombres_66,Hombres_67,Hombres_68,Hombres_69,Hombres_70,Hombres_71,Hombres_72,Hombres_73,Hombres_74,Hombres_75,Hombres_76,Hombres_77,Hombres_78,Hombres_79,Hombres_80,Hombres_81,Hombres_82,Hombres_83,Hombres_84,Hombres_85 y más,Mujeres_0,Mujeres_1,Mujeres_2,Mujeres_3,Mujeres_4,Mujeres_5,Mujeres_6,Mujeres_7,Mujeres_8,Mujeres_9,Mujeres_10,Mujeres_11,Mujeres_12,Mujeres_13,Mujeres_14,Mujeres_15,Mujeres_16,Mujeres_17,Mujeres_18,Mujeres_19,Mujeres_20,Mujeres_21,Mujeres_22,Mujeres_23,Mujeres_24,Mujeres_25,Mujeres_26,Mujeres_27,Mujeres_28,Mujeres_29,Mujeres_30,Mujeres_31,Mujeres_32,Mujeres_33,Mujeres_34,Mujeres_35,Mujeres_36,Mujeres_37,Mujeres_38,Mujeres_39,Mujeres_40,Mujeres_41,Mujeres_42,Mujeres_43,Mujeres_44,Mujeres_45,Mujeres_46,Mujeres_47,Mujeres_48,Mujeres_49,Mujeres_50,Mujeres_51,Mujeres_52,Mujeres_53,Mujeres_54,Mujeres_55,Mujeres_56,Mujeres_57,Mujeres_58,Mujeres_59,Mujeres_60,Mujeres_61,Mujeres_62,Mujeres_63,Mujeres_64,Mujeres_65,Mujeres_66,Mujeres_67,Mujeres_68,Mujeres_69,Mujeres_70,Mujeres_71,Mujeres_72,Mujeres_73,Mujeres_74,Mujeres_75,Mujeres_76,Mujeres_77,Mujeres_78,Mujeres_79,Mujeres_80,Mujeres_81,Mujeres_82,Mujeres_83,Mujeres_84,Mujeres_85 y más,Total_0,Total_1,Total_2,Total_3,Total_4,Total_5,Total_6,Total_7,Total_8,Total_9,Total_10,Total_11,Total_12,Total_13,Total_14,Total_15,Total_16,Total_17,Total_18,Total_19,Total_20,Total_21,Total_22,Total_23,Total_24,Total_25,Total_26,Total_27,Total_28,Total_29,Total_30,Total_31,Total_32,Total_33,Total_34,Total_35,Total_36,Total_37,Total_38,Total_39,Total_40,Total_41,Total_42,Total_43,Total_44,Total_45,Total_46,Total_47,Total_48,Total_49,Total_50,Total_51,Total_52,Total_53,Total_54,Total_55,Total_56,Total_57,Total_58,Total_59,Total_60,Total_61,Total_62,Total_63,Total_64,Total_65,Total_66,Total_67,Total_68,Total_69,Total_70,Total_71,Total_72,Total_73,Total_74,Total_75,Total_76,Total_77,Total_78,Total_79,Total_80,Total_81,Total_82,Total_83,Total_84,Total_85 y más,Total Hombres,Total Mujeres,Total
15,5,ANTIOQUIA,5001,MEDELLÍN,2025,Cabecera Municipal,12913,13092,13234,13337,13887,14579,14837,15076,15223,15344,15457,15577,15718,15901,16139,16432,16803,17292,17892,18521,19255,19982,20751,21490,22185,22869,23500,23917,24146,24161,23995,23657,23239,22766,22214,21525,20802,20108,19435,18831,18332,17908,17474,17073,16574,15949,15208,14482,13807,13306,13017,12906,12857,12853,12881,12982,13083,13165,13184,13085,12830,12451,12010,11548,11051,10555,10046,9514,8972,8401,7852,7313,6774,6259,5741,5252,4789,4328,3894,3475,3073,2705,2357,2032,1752,8525,12234,12484,12678,12842,13377,14030,14297,14560,14663,14780,14909,15078,15247,15474,15723,16032,16410,16885,17481,18160,18878,19633,20389,21112,21818,22517,23138,23556,23793,23832,23689,23397,23067,22749,22364,21899,21378,20896,20436,20085,19862,19730,19598,19444,19171,18709,18110,17524,16980,16604,16434,16472,16563,16710,16900,17152,17439,17675,17841,17853,17659,17307,16869,16355,15795,15221,14620,13949,13261,12573,11864,11173,10468,9783,9096,8454,7822,7208,6604,6022,5413,4856,4297,3795,3312,17318,25147,25576,25912,26179,27264,28609,29134,29636,29886,30124,30366,30655,30965,31375,31862,32464,33213,34177,35373,36681,38133,39615,41140,42602,44003,45386,46638,47473,47939,47993,47684,47054,46306,45515,44578,43424,42

## Limpieza de BD Proyección Etnia

In [771]:
#Cambiamos nombre a columnas departamento y municipio
etn = etn.rename(columns={
    'COD_DPTO-MPIO': 'MPIO'
})
#Eliminamos municipio 94663 - Guainía - Mapiripana
etn = etn[etn['MPIO'] != 94663]
#Información después del 2025
etn = etn[etn['AÑO'] >= 2025]
etn.head(5)

,COD_DPTO,DEPARTAMENTO,MPIO,MUNICIPIO,AÑO,ÁREA GEOGRÁFICA,Total,Indígena,Gitano(a) o Rrom,"Raizal del Archipiélago de San Andrés, Providencia y Santa Catalina",Palenquero(a) de San Basilio,"Negro(a), mulato(a), afrodescendiente, afrocolombiano(a)",Ningún grupo étnico-racial
21,5,Antioquia,5001,Medellín,2025,Cabecera Municipal,2593542,2293,54,353,70,66273,2524499
22,5,Antioquia,5001,Medellín,2025,Centros Poblados y Rural Disperso,41028,38,0,1,1,253,40735
23,5,Antioquia,5001,Medellín,2025,Total,2634570,2331,54,354,71,66526,2565234
24,5,Antioquia,5001,Medellín,2026,Cabecera Municipal,2609841,2307,54,355,70,66690,2540365
25,5,Antioquia,5001,Medellín,2026,Centros Poblados y Rural Disperso,40821,37,0,1,1,252,40530


# Vistas en SQL, agrupaciones y BD con info completa

In [826]:
# Conectar a sqlite
conn = sqlite3.connect("iets.sqlite")

# Guardar en SQL
municipios.to_sql("mun", conn, if_exists="replace", index=False)
prestadores.to_sql("prest", conn, if_exists="replace", index=False)
pob.to_sql("pob", conn, if_exists="replace", index=False)
etn.to_sql("etn", conn, if_exists="replace", index=False)

36993

In [827]:
# Listar todas las vistas
query = "SELECT name FROM sqlite_master WHERE type='view';"
views = conn.execute(query).fetchall()

# views es una lista de tuplas, extraemos solo los nombres
view_names = [v[0] for v in views]

# Eliminar cada vista
for view_name in view_names:
    drop_query = f"DROP VIEW IF EXISTS {view_name};"
    conn.execute(drop_query)

conn.commit()
print(f"Se eliminaron {len(view_names)} vistas.")

Se eliminaron 6 vistas.


## Vistas info DANE + Municipios

In [828]:
#Agrupación por área geográfica de hombre y mujeres y grupos de edades
query = """
CREATE VIEW IF NOT EXISTS pobhm AS
SELECT 
    p.MPIO,
    p.DPNOM AS depa_nombre,
    p.DPMP AS muni_nombre,
    p."AÑO",

    -- Totales por área geográfica
    SUM(CASE WHEN p."ÁREA GEOGRÁFICA" = 'Total' THEN p.Total ELSE 0 END) AS total_pob,
    SUM(CASE WHEN p."ÁREA GEOGRÁFICA" = 'Cabecera Municipal' THEN p.Total ELSE 0 END) AS total_cabecera,
    SUM(CASE WHEN p."ÁREA GEOGRÁFICA" = 'Centros Poblados y Rural Disperso' THEN p.Total ELSE 0 END) AS total_cprd,
    

    -- Totales de hombres
    SUM(CASE WHEN p."ÁREA GEOGRÁFICA" = 'Cabecera Municipal' THEN p."Total Hombres" ELSE 0 END) AS hombres_cabecera,
    SUM(CASE WHEN p."ÁREA GEOGRÁFICA" = 'Centros Poblados y Rural Disperso' THEN p."Total Hombres" ELSE 0 END) AS hombres_cprd,
    SUM(CASE WHEN p."ÁREA GEOGRÁFICA" = 'Total' THEN p."Total Hombres" ELSE 0 END) AS hombres_total,

    -- Totales de mujeres
    SUM(CASE WHEN p."ÁREA GEOGRÁFICA" = 'Cabecera Municipal' THEN p."Total Mujeres" ELSE 0 END) AS mujeres_cabecera,
    SUM(CASE WHEN p."ÁREA GEOGRÁFICA" = 'Centros Poblados y Rural Disperso' THEN p."Total Mujeres" ELSE 0 END) AS mujeres_cprd,
    SUM(CASE WHEN p."ÁREA GEOGRÁFICA" = 'Total' THEN p."Total Mujeres" ELSE 0 END) AS mujeres_total,

    -- Grupos de edad - HOMBRES
    SUM(CASE WHEN p."ÁREA GEOGRÁFICA" = 'Total' THEN p.Hombres_0 + p.Hombres_1 + p.Hombres_2 + p.Hombres_3 + p.Hombres_4 + p.Hombres_5 ELSE 0 END) AS hombres_primera_infancia,
    SUM(CASE WHEN p."ÁREA GEOGRÁFICA" = 'Total' THEN p.Hombres_6 + p.Hombres_7 + p.Hombres_8 + p.Hombres_9 + p.Hombres_10 + p.Hombres_11 ELSE 0 END) AS hombres_infancia,
    SUM(CASE WHEN p."ÁREA GEOGRÁFICA" = 'Total' THEN p.Hombres_12 + p.Hombres_13 + p.Hombres_14 + p.Hombres_15 + p.Hombres_16 + p.Hombres_17 ELSE 0 END) AS hombres_adolescencia,
    SUM(CASE WHEN p."ÁREA GEOGRÁFICA" = 'Total' THEN p.Hombres_18 + p.Hombres_19 + p.Hombres_20 + p.Hombres_21 + 
        p.Hombres_22 + p.Hombres_23 + p.Hombres_24 + p.Hombres_25 + p.Hombres_26 ELSE 0 END) AS hombres_juventud,
    SUM(CASE WHEN p."ÁREA GEOGRÁFICA" = 'Total' THEN p.Hombres_27 + p.Hombres_28 + p.Hombres_29 + p.Hombres_30 + p.Hombres_31 + p.Hombres_32 + p.Hombres_33 + p.Hombres_34 + 
        p.Hombres_35 + p.Hombres_36 + p.Hombres_37 + p.Hombres_38 + p.Hombres_39 + p.Hombres_40 + p.Hombres_41 + p.Hombres_42 + 
        p.Hombres_43 + p.Hombres_44 + p.Hombres_45 + p.Hombres_46 + p.Hombres_47 + p.Hombres_48 + p.Hombres_49 + p.Hombres_50 + 
        p.Hombres_51 + p.Hombres_52 + p.Hombres_53 + p.Hombres_54 + p.Hombres_55 + p.Hombres_56 + p.Hombres_57 + p.Hombres_58 + 
        p.Hombres_59 ELSE 0 END) AS hombres_adultez,
    SUM(CASE WHEN p."ÁREA GEOGRÁFICA" = 'Total' THEN p.Hombres_60 + p.Hombres_61 + p.Hombres_62 + p.Hombres_63 + p.Hombres_64 + p.Hombres_65 + p.Hombres_66 + p.Hombres_67 + 
        p.Hombres_68 + p.Hombres_69 + p.Hombres_70 + p.Hombres_71 + p.Hombres_72 + p.Hombres_73 + p.Hombres_74 + p.Hombres_75 + 
        p.Hombres_76 + p.Hombres_77 + p.Hombres_78 + p.Hombres_79 + p.Hombres_80 + p.Hombres_81 + p.Hombres_82 + p.Hombres_83 + 
        p.Hombres_84 + p."Hombres_85 y más" ELSE 0 END) AS hombres_vejez,

    -- Grupos de edad - MUJERES
    SUM(CASE WHEN p."ÁREA GEOGRÁFICA" = 'Total' THEN p.Mujeres_0 + p.Mujeres_1 + p.Mujeres_2 + p.Mujeres_3 + p.Mujeres_4 + p.Mujeres_5 ELSE 0 END) AS mujeres_primera_infancia,
    SUM(CASE WHEN p."ÁREA GEOGRÁFICA" = 'Total' THEN p.Mujeres_6 + p.Mujeres_7 + p.Mujeres_8 + p.Mujeres_9 + p.Mujeres_10 + p.Mujeres_11 ELSE 0 END) AS mujeres_infancia,
    SUM(CASE WHEN p."ÁREA GEOGRÁFICA" = 'Total' THEN p.Mujeres_12 + p.Mujeres_13 + p.Mujeres_14 + p.Mujeres_15 + p.Mujeres_16 + p.Mujeres_17 ELSE 0 END) AS mujeres_adolescencia,
    SUM(CASE WHEN p."ÁREA GEOGRÁFICA" = 'Total' THEN p.Mujeres_18 + p.Mujeres_19 + p.Mujeres_20 + p.Mujeres_21 + 
        p.Mujeres_22 + p.Mujeres_23 + p.Mujeres_24 + p.Mujeres_25 + p.Mujeres_26 ELSE 0 END) AS mujeres_juventud,
    SUM(CASE WHEN p."ÁREA GEOGRÁFICA" = 'Total' THEN p.Mujeres_27 + p.Mujeres_28 + p.Mujeres_29 + p.Mujeres_30 + p.Mujeres_31 + p.Mujeres_32 + p.Mujeres_33 + p.Mujeres_34 + 
        p.Mujeres_35 + p.Mujeres_36 + p.Mujeres_37 + p.Mujeres_38 + p.Mujeres_39 + p.Mujeres_40 + p.Mujeres_41 + p.Mujeres_42 + 
        p.Mujeres_43 + p.Mujeres_44 + p.Mujeres_45 + p.Mujeres_46 + p.Mujeres_47 + p.Mujeres_48 + p.Mujeres_49 + p.Mujeres_50 + 
        p.Mujeres_51 + p.Mujeres_52 + p.Mujeres_53 + p.Mujeres_54 + p.Mujeres_55 + p.Mujeres_56 + p.Mujeres_57 + p.Mujeres_58 + 
        p.Mujeres_59 ELSE 0 END) AS mujeres_adultez,
    SUM(CASE WHEN p."ÁREA GEOGRÁFICA" = 'Total' THEN p.Mujeres_60 + p.Mujeres_61 + p.Mujeres_62 + p.Mujeres_63 + p.Mujeres_64 + p.Mujeres_65 + p.Mujeres_66 + p.Mujeres_67 + 
        p.Mujeres_68 + p.Mujeres_69 + p.Mujeres_70 + p.Mujeres_71 + p.Mujeres_72 + p.Mujeres_73 + p.Mujeres_74 + p.Mujeres_75 + 
        p.Mujeres_76 + p.Mujeres_77 + p.Mujeres_78 + p.Mujeres_79 + p.Mujeres_80 + p.Mujeres_81 + p.Mujeres_82 + p.Mujeres_83 + 
        p.Mujeres_84 + p."Mujeres_85 y más" ELSE 0 END) AS mujeres_vejez

FROM pob p
GROUP BY p.MPIO, p."AÑO";
"""

conn.execute(query)
conn.commit()



In [829]:
df = pd.read_sql_query("SELECT * FROM pobhm WHERE AÑO = 2025", conn)
df

,MPIO,depa_nombre,muni_nombre,AÑO,total_pob,total_cabecera,total_cprd,hombres_cabecera,hombres_cprd,hombres_total,mujeres_cabecera,mujeres_cprd,mujeres_total,hombres_primera_infancia,hombres_infancia,hombres_adolescencia,hombres_juventud,hombres_adultez,hombres_vejez,mujeres_primera_infancia,mujeres_infancia,mujeres_adolescencia,mujeres_juventud,mujeres_adultez,mujeres_vejez
0,5001,ANTIOQUIA,MEDELLÍN,2025,2634570,2593542,41028,1219707,20731,1240438,1373835,20297,1394132,82633,93138,99872,189152,588771,186872,79227,89761,97165,185718,657696,284565
1,5002,ANTIOQUIA,ABEJORRAL,2025,21622,9369,12253,4599,6700,11299,4770,5553,10323,719,925,1006,1322,4919,2408,708,841,854,1074,4459,2387
2,5004,ANTIOQUIA,ABRIAQUÍ,2025,2872,1029,1843,559,1015,1574,470,828,1298,128,135,143,209,652,307,108,112,106,143,579,250
3,5021,ANTIOQUIA,ALEJANDRÍA,2025,4989,2960,2029,1380,1087,2467,1580,942,2522,232,231,210,288,1026,480,228,217,185,262,1077,553
4,5030,ANTIOQUIA,AMAGÁ,2025,32628,17156,15472,8192,7809,16001,8964,7663,16627,1132,1257,1305,2210,7425,2672,1089,1222,1279,2115,7853,3069
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1116,97889,VAUPÉS,YAVARATÉ,2025,1265,0,1265,0,658,658,0,607,607,69,136,156,140,133,24,74,141,147,99,102,44
1117,99001,VICHADA,PUERTO CARREÑO,2025,22963,17605,5358,8827,2829,11656,8778,2529,11307,1187,1121,1239,2218,4855,1036,1140,1098,1155,2018,4920,976
1118,99524,VICHADA,LA PRIMAVERA,2025,11380,7020,4360,3515,2346,5861,3505,2014,5519,589,635,721,1007,2315,594,536,629,669,926,2246,513
1119,99624,VICHADA,SANTA ROSALÍA,2025,4732,2935,1797,1438,960,2398,1497,837,2334,211,281,294,378,1029,205,226,293,301,375,905,234


In [830]:
#Agrupación del total de población por pertenecia étnica y por área geográfica

query = """
CREATE VIEW IF NOT EXISTS pobetn AS
SELECT 
    e.MPIO,
    e."AÑO",

    -- Personas con pertenencia étnica por área geográfica
    SUM(CASE WHEN e."ÁREA GEOGRÁFICA" = 'Total' THEN e."Total" ELSE 0 END) AS total_pob,
    SUM(CASE WHEN e."ÁREA GEOGRÁFICA" = 'Cabecera Municipal' THEN e."Total" - e."Ningún grupo étnico-racial" ELSE 0 END) AS etnia_cabecera,
    SUM(CASE WHEN e."ÁREA GEOGRÁFICA" = 'Centros Poblados y Rural Disperso' THEN e."Total" - e."Ningún grupo étnico-racial" ELSE 0 END) AS etnia_cprd,
    SUM(CASE WHEN e."ÁREA GEOGRÁFICA" = 'Total' THEN e."Total" - e."Ningún grupo étnico-racial" ELSE 0 END) AS etnia_total
    

FROM etn e

GROUP BY e.MPIO, e."AÑO";
"""
conn.execute(query)
conn.commit()



In [831]:
df = pd.read_sql_query("SELECT * FROM pobetn WHERE AÑO = 2025", conn)
df

,MPIO,AÑO,total_pob,etnia_cabecera,etnia_cprd,etnia_total
0,5001,2025,2634570,69043,293,69336
1,5002,2025,21622,78,8,86
2,5004,2025,2872,35,26,61
3,5021,2025,4989,15,1,16
4,5030,2025,32628,163,18,181
...,...,...,...,...,...,...
1116,97889,2025,1265,0,1164,1164
1117,99001,2025,22963,3631,2940,6571
1118,99524,2025,11380,497,1881,2378
1119,99624,2025,4732,376,881,1257


In [832]:
#Creación de la vista de datos poblacionales completos por municipio: pobhm + pobetn
query="""
CREATE VIEW data_pob AS
SELECT 
    pobhm.*,
    pobetn.etnia_cabecera,
    pobetn.etnia_cprd,
    pobetn.etnia_total
FROM pobhm
JOIN pobetn
    ON pobhm.MPIO = pobetn.MPIO    
    AND pobhm.AÑO = pobetn.AÑO
"""

conn.execute(query)
conn.commit()

In [833]:
df = pd.read_sql_query("SELECT * FROM data_pob WHERE AÑO = 2025", conn)
df

,MPIO,depa_nombre,muni_nombre,AÑO,total_pob,total_cabecera,total_cprd,hombres_cabecera,hombres_cprd,hombres_total,mujeres_cabecera,mujeres_cprd,mujeres_total,hombres_primera_infancia,hombres_infancia,hombres_adolescencia,hombres_juventud,hombres_adultez,hombres_vejez,mujeres_primera_infancia,mujeres_infancia,mujeres_adolescencia,mujeres_juventud,mujeres_adultez,mujeres_vejez,etnia_cabecera,etnia_cprd,etnia_total
0,5001,ANTIOQUIA,MEDELLÍN,2025,2634570,2593542,41028,1219707,20731,1240438,1373835,20297,1394132,82633,93138,99872,189152,588771,186872,79227,89761,97165,185718,657696,284565,69043,293,69336
1,5002,ANTIOQUIA,ABEJORRAL,2025,21622,9369,12253,4599,6700,11299,4770,5553,10323,719,925,1006,1322,4919,2408,708,841,854,1074,4459,2387,78,8,86
2,5004,ANTIOQUIA,ABRIAQUÍ,2025,2872,1029,1843,559,1015,1574,470,828,1298,128,135,143,209,652,307,108,112,106,143,579,250,35,26,61
3,5021,ANTIOQUIA,ALEJANDRÍA,2025,4989,2960,2029,1380,1087,2467,1580,942,2522,232,231,210,288,1026,480,228,217,185,262,1077,553,15,1,16
4,5030,ANTIOQUIA,AMAGÁ,2025,32628,17156,15472,8192,7809,16001,8964,7663,16627,1132,1257,1305,2210,7425,2672,1089,1222,1279,2115,7853,3069,163,18,181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1116,97889,VAUPÉS,YAVARATÉ,2025,1265,0,1265,0,658,658,0,607,607,69,136,156,140,133,24,74,141,147,99,102,44,0,1164,1164
1117,99001,VICHADA,PUERTO CARREÑO,2025,22963,17605,5358,8827,2829,11656,8778,2529,11307,1187,1121,1239,2218,4855,1036,1140,1098,1155,2018,4920,976,3631,2940,6571
1118,99524,VICHADA,LA PRIMAVERA,2025,11380,7020,4360,3515,2346,5861,3505,2014,5519,589,635,721,1007,2315,594,536,629,669,926,2246,513,497,1881,2378
1119,99624,VICHADA,SANTA ROSALÍA,2025,4732,2935,1797,1438,960,2398,1497,837,2334,211,281,294,378,1029,205,226,293,301,375,905,234,376,881,1257


In [835]:
#Creación de tabla con todos los datos del municipio: data_pob + mun (Municipios)
#Unión entre la BD de municipios, se trae la información de región y superficie y las vistas de la información del DANE data_pob
query= """
CREATE VIEW data_pob_g AS
SELECT 
    dp.*,
    m.Superficie AS superficie, 
    m.Region AS region
FROM data_pob dp
LEFT JOIN mun m 
    ON dp.MPIO = m.MPIO;

"""

conn.execute(query)
conn.commit()

In [839]:
# Leer la vista mun1 como DataFrame
df = pd.read_sql_query("SELECT * FROM data_pob_g WHERE AÑO = 2025", conn)
df

,MPIO,depa_nombre,muni_nombre,AÑO,total_pob,total_cabecera,total_cprd,hombres_cabecera,hombres_cprd,hombres_total,mujeres_cabecera,mujeres_cprd,mujeres_total,hombres_primera_infancia,hombres_infancia,hombres_adolescencia,hombres_juventud,hombres_adultez,hombres_vejez,mujeres_primera_infancia,mujeres_infancia,mujeres_adolescencia,mujeres_juventud,mujeres_adultez,mujeres_vejez,etnia_cabecera,etnia_cprd,etnia_total,superficie,region
0,5001,ANTIOQUIA,MEDELLÍN,2025,2634570,2593542,41028,1219707,20731,1240438,1373835,20297,1394132,82633,93138,99872,189152,588771,186872,79227,89761,97165,185718,657696,284565,69043,293,69336,350.666623,Región Eje Cafetero
1,5002,ANTIOQUIA,ABEJORRAL,2025,21622,9369,12253,4599,6700,11299,4770,5553,10323,719,925,1006,1322,4919,2408,708,841,854,1074,4459,2387,78,8,86,497.566212,Región Eje Cafetero
2,5004,ANTIOQUIA,ABRIAQUÍ,2025,2872,1029,1843,559,1015,1574,470,828,1298,128,135,143,209,652,307,108,112,106,143,579,250,35,26,61,287.641603,Región Eje Cafetero
3,5021,ANTIOQUIA,ALEJANDRÍA,2025,4989,2960,2029,1380,1087,2467,1580,942,2522,232,231,210,288,1026,480,228,217,185,262,1077,553,15,1,16,119.467683,Región Eje Cafetero
4,5030,ANTIOQUIA,AMAGÁ,2025,32628,17156,15472,8192,7809,16001,8964,7663,16627,1132,1257,1305,2210,7425,2672,1089,1222,1279,2115,7853,3069,163,18,181,89.245903,Región Eje Cafetero
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1116,97889,VAUPÉS,YAVARATÉ,2025,1265,0,1265,0,658,658,0,607,607,69,136,156,140,133,24,74,141,147,99,102,44,0,1164,1164,4791.180820,Región Llano
1117,99001,VICHADA,PUERTO CARREÑO,2025,22963,17605,5358,8827,2829,11656,8778,2529,11307,1187,1121,1239,2218,4855,1036,1140,1098,1155,2018,4920,976,3631,2940,6571,12175.639571,Región Llano
1118,99524,VICHADA,LA PRIMAVERA,2025,11380,7020,4360,3515,2346,5861,3505,2014,5519,589,635,721,1007,2315,594,536,629,669,926,2246,513,497,1881,2378,18188.762851,Región Llano
1119,99624,VICHADA,SANTA ROSALÍA,2025,4732,2935,1797,1438,960,2398,1497,837,2334,211,281,294,378,1029,205,226,293,301,375,905,234,376,881,1257,3930.629399,Región Llano


In [840]:
# Leer la vista como DataFrame
df = pd.read_sql_query("SELECT * FROM data_pob_g WHERE superficie IS NULL AND AÑO = 2025", conn)
df

,MPIO,depa_nombre,muni_nombre,AÑO,total_pob,total_cabecera,total_cprd,hombres_cabecera,hombres_cprd,hombres_total,mujeres_cabecera,mujeres_cprd,mujeres_total,hombres_primera_infancia,hombres_infancia,hombres_adolescencia,hombres_juventud,hombres_adultez,hombres_vejez,mujeres_primera_infancia,mujeres_infancia,mujeres_adolescencia,mujeres_juventud,mujeres_adultez,mujeres_vejez,etnia_cabecera,etnia_cprd,etnia_total,superficie,region
0,13490,BOLÍVAR,NOROSÍ,2025,10957,2390,8567,1207,4605,5812,1183,3962,5145,752,702,567,855,2451,485,757,683,546,858,1953,348,232,43,275,None,None
1,19300,CAUCA,GUACHENÉ,2025,20785,6534,14251,2932,6680,9612,3602,7571,11173,780,927,1008,1551,4148,1198,798,912,1020,1761,5219,1463,6429,13547,19976,None,None
2,23682,CÓRDOBA,SAN JOSÉ DE URÉ,2025,14830,5438,9392,2658,5098,7756,2780,4294,7074,960,1022,961,1144,2767,902,911,942,889,1102,2547,683,3430,7413,10843,None,None
3,23815,CÓRDOBA,TUCHÍN,2025,57361,7314,50047,3592,26067,29659,3722,23980,27702,4079,3865,3446,4940,10394,2935,3693,3587,3268,4655,9518,2981,7195,49811,57006,None,None


In [857]:
conn.execute("DROP VIEW IF EXISTS data_pob_g")


In [858]:
#Añadir valores a los 4 municipios vacios y también se cam
query = """
CREATE VIEW data_pob_g AS
SELECT
    dp.*,
    CASE 
        WHEN m.Superficie IS NOT NULL THEN m.Superficie
        WHEN dp.MPIO = 13490 THEN 5000      -- Bolívar - Norosí
        WHEN dp.MPIO = 19300 THEN 1200      -- Cauca - Guachené
        WHEN dp.MPIO = 23682 THEN 1800      -- Córdoba - San José de Uré
        WHEN dp.MPIO = 23815 THEN 1000      -- Córdoba - Tuchín
        ELSE NULL
    END AS superficie,
    CASE 
        WHEN m.Region IS NOT NULL THEN m.Region
        WHEN dp.MPIO = 13490 THEN 'Región Llano'
        WHEN dp.MPIO = 19300 THEN 'Región Pacífico'
        WHEN dp.MPIO = 23682 THEN 'Región Caribe'
        WHEN dp.MPIO = 23815 THEN 'Región Caribe'
        ELSE NULL
    END AS region
FROM data_pob dp
LEFT JOIN mun m ON dp.MPIO = m.MPIO;
"""

conn.execute(query)
conn.commit()

In [862]:
# Leer la vista como DataFrame - debería salir cero
df = pd.read_sql_query("SELECT * FROM data_pob_g", conn)
df

,MPIO,depa_nombre,muni_nombre,AÑO,total_pob,total_cabecera,total_cprd,hombres_cabecera,hombres_cprd,hombres_total,mujeres_cabecera,mujeres_cprd,mujeres_total,hombres_primera_infancia,hombres_infancia,hombres_adolescencia,hombres_juventud,hombres_adultez,hombres_vejez,mujeres_primera_infancia,mujeres_infancia,mujeres_adolescencia,mujeres_juventud,mujeres_adultez,mujeres_vejez,etnia_cabecera,etnia_cprd,etnia_total,superficie,region
0,5001,ANTIOQUIA,MEDELLÍN,2025,2634570,2593542,41028,1219707,20731,1240438,1373835,20297,1394132,82633,93138,99872,189152,588771,186872,79227,89761,97165,185718,657696,284565,69043,293,69336,350.666623,Región Eje Cafetero
1,5001,ANTIOQUIA,MEDELLÍN,2026,2650662,2609841,40821,1227289,20637,1247926,1382552,20184,1402736,81133,92602,99544,185641,595878,193128,77685,89178,96698,181833,662186,295156,69476,291,69767,350.666623,Región Eje Cafetero
2,5001,ANTIOQUIA,MEDELLÍN,2027,2665474,2624783,40691,1234291,20567,1254858,1390492,20124,1410616,80126,91404,99335,182426,602358,199209,76575,88003,96301,178278,666018,305441,69874,290,70164,350.666623,Región Eje Cafetero
3,5001,ANTIOQUIA,MEDELLÍN,2028,2679376,2638734,40642,1240910,20530,1261440,1397824,20112,1417936,79481,89705,99201,179630,608401,205022,75783,86339,95967,175175,669374,315298,70246,290,70536,350.666623,Región Eje Cafetero
4,5001,ANTIOQUIA,MEDELLÍN,2029,2692600,2651875,40725,1247211,20554,1267765,1404664,20171,1424835,78735,87988,99084,177273,614137,210548,74912,84607,95661,172554,672417,324684,70595,290,70885,350.666623,Región Eje Cafetero
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5600,99773,VICHADA,CUMARIBO,2025,88392,3102,85290,1544,44614,46158,1558,40676,42234,6452,6390,6395,8475,15202,3244,6285,6171,6101,7667,13461,2549,1273,77907,79180,64509.421614,Región Llano
5601,99773,VICHADA,CUMARIBO,2026,89677,3117,86560,1542,45145,46687,1575,41415,42990,6459,6325,6375,8542,15606,3380,6289,6138,6119,7811,13943,2690,1279,79067,80346,64509.421614,Región Llano
5602,99773,VICHADA,CUMARIBO,2027,90998,3110,87888,1531,45691,47222,1579,42197,43776,6488,6259,6350,8606,16016,3503,6311,6107,6128,7936,14454,2840,1277,80280,81557,64509.421614,Región Llano
5603,99773,VICHADA,CUMARIBO,2028,92158,3106,89052,1536,46167,47703,1570,42885,44455,6526,6193,6297,8644,16411,3632,6337,6066,6112,8033,14925,2982,1275,81343,82618,64509.421614,Región Llano


In [860]:
# Leer la vista como DataFrame - debería salir cero
df = pd.read_sql_query("SELECT * FROM data_pob_g WHERE superficie IS NULL AND AÑO = 2025", conn)
df

,MPIO,depa_nombre,muni_nombre,AÑO,total_pob,total_cabecera,total_cprd,hombres_cabecera,hombres_cprd,hombres_total,mujeres_cabecera,mujeres_cprd,mujeres_total,hombres_primera_infancia,hombres_infancia,hombres_adolescencia,hombres_juventud,hombres_adultez,hombres_vejez,mujeres_primera_infancia,mujeres_infancia,mujeres_adolescencia,mujeres_juventud,mujeres_adultez,mujeres_vejez,etnia_cabecera,etnia_cprd,etnia_total,superficie,region


In [861]:
df = pd.read_sql_query("SELECT * FROM data_pob_g WHERE AÑO = 2025", conn)
df

,MPIO,depa_nombre,muni_nombre,AÑO,total_pob,total_cabecera,total_cprd,hombres_cabecera,hombres_cprd,hombres_total,mujeres_cabecera,mujeres_cprd,mujeres_total,hombres_primera_infancia,hombres_infancia,hombres_adolescencia,hombres_juventud,hombres_adultez,hombres_vejez,mujeres_primera_infancia,mujeres_infancia,mujeres_adolescencia,mujeres_juventud,mujeres_adultez,mujeres_vejez,etnia_cabecera,etnia_cprd,etnia_total,superficie,region
0,5001,ANTIOQUIA,MEDELLÍN,2025,2634570,2593542,41028,1219707,20731,1240438,1373835,20297,1394132,82633,93138,99872,189152,588771,186872,79227,89761,97165,185718,657696,284565,69043,293,69336,350.666623,Región Eje Cafetero
1,5002,ANTIOQUIA,ABEJORRAL,2025,21622,9369,12253,4599,6700,11299,4770,5553,10323,719,925,1006,1322,4919,2408,708,841,854,1074,4459,2387,78,8,86,497.566212,Región Eje Cafetero
2,5004,ANTIOQUIA,ABRIAQUÍ,2025,2872,1029,1843,559,1015,1574,470,828,1298,128,135,143,209,652,307,108,112,106,143,579,250,35,26,61,287.641603,Región Eje Cafetero
3,5021,ANTIOQUIA,ALEJANDRÍA,2025,4989,2960,2029,1380,1087,2467,1580,942,2522,232,231,210,288,1026,480,228,217,185,262,1077,553,15,1,16,119.467683,Región Eje Cafetero
4,5030,ANTIOQUIA,AMAGÁ,2025,32628,17156,15472,8192,7809,16001,8964,7663,16627,1132,1257,1305,2210,7425,2672,1089,1222,1279,2115,7853,3069,163,18,181,89.245903,Región Eje Cafetero
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1116,97889,VAUPÉS,YAVARATÉ,2025,1265,0,1265,0,658,658,0,607,607,69,136,156,140,133,24,74,141,147,99,102,44,0,1164,1164,4791.180820,Región Llano
1117,99001,VICHADA,PUERTO CARREÑO,2025,22963,17605,5358,8827,2829,11656,8778,2529,11307,1187,1121,1239,2218,4855,1036,1140,1098,1155,2018,4920,976,3631,2940,6571,12175.639571,Región Llano
1118,99524,VICHADA,LA PRIMAVERA,2025,11380,7020,4360,3515,2346,5861,3505,2014,5519,589,635,721,1007,2315,594,536,629,669,926,2246,513,497,1881,2378,18188.762851,Región Llano
1119,99624,VICHADA,SANTA ROSALÍA,2025,4732,2935,1797,1438,960,2398,1497,837,2334,211,281,294,378,1029,205,226,293,301,375,905,234,376,881,1257,3930.629399,Región Llano


## Vista Prestadores

In [863]:
query = """
CREATE VIEW prest_agr AS
SELECT
  depa_nombre,
  muni_nombre,

  COUNT(*) AS total_prestadores,

  -- Conteo por tipo clpr_nombre usando CASE
  SUM(CASE WHEN clpr_nombre = 'Profesional Independiente' THEN 1 ELSE 0 END) AS profesional_independiente,
  SUM(CASE WHEN clpr_nombre = 'Instituciones Prestadoras de Servicios de Salud - IPS' THEN 1 ELSE 0 END) AS ips,
  SUM(CASE WHEN clpr_nombre = 'Objeto Social Diferente a la Prestación de Servicios de Salud' THEN 1 ELSE 0 END) AS objeto_social_diferente,
  SUM(CASE WHEN clpr_nombre = 'Transporte Especial de Pacientes' THEN 1 ELSE 0 END) AS transporte_especial,

  -- Conteo por clase_persona
  SUM(CASE WHEN clase_persona = 'NATURAL' THEN 1 ELSE 0 END) AS natural,
  SUM(CASE WHEN clase_persona = 'JURIDICO' THEN 1 ELSE 0 END) AS juridico,

  -- Conteo por naju_nombre
  SUM(CASE WHEN naju_nombre = 'Privada' THEN 1 ELSE 0 END) AS privada,
  SUM(CASE WHEN naju_nombre = 'Mixta' THEN 1 ELSE 0 END) AS mixta,
  SUM(CASE WHEN naju_nombre = 'Pública' THEN 1 ELSE 0 END) AS publica,

  -- Conteo por año de fecha_vencimiento
  SUM(CASE WHEN SUBSTR(CAST(fecha_vencimiento AS TEXT), 1, 4) = '2025' THEN 1 ELSE 0 END) AS vencimiento_2025,
  SUM(CASE WHEN SUBSTR(CAST(fecha_vencimiento AS TEXT), 1, 4) = '2026' THEN 1 ELSE 0 END) AS vencimiento_2026,
  SUM(CASE WHEN SUBSTR(CAST(fecha_vencimiento AS TEXT), 1, 4) = '2027' THEN 1 ELSE 0 END) AS vencimiento_2027,
  SUM(CASE WHEN SUBSTR(CAST(fecha_vencimiento AS TEXT), 1, 4) = '2028' THEN 1 ELSE 0 END) AS vencimiento_2028,
  SUM(CASE WHEN SUBSTR(CAST(fecha_vencimiento AS TEXT), 1, 4) = '2029' THEN 1 ELSE 0 END) AS vencimiento_2029,

  -- Columnas con valores representativos
  MIN("Municipio PDET") AS municipio_pdet,
  MIN("Municipio ZOMAC") AS municipio_zomac,
  MIN("Municipio PNIS") AS municipio_pnis,
  MIN("Municipio PNSR antes 2023") AS municipio_pnsr_antes_2023,
  MIN("Municipio PNSR 2023") AS municipio_pnsr_2023,
  MIN("Municipio PNSR 2024") AS municipio_pnsr_2024

FROM prest
GROUP BY depa_nombre, muni_nombre;

"""

conn.execute(query)
conn.commit()



In [864]:
# Leer la vista prest_pob_2025 como DataFrame
df = pd.read_sql_query("SELECT * FROM prest_agr", conn)
df

,depa_nombre,muni_nombre,total_prestadores,profesional_independiente,ips,objeto_social_diferente,transporte_especial,natural,juridico,privada,mixta,publica,vencimiento_2025,vencimiento_2026,vencimiento_2027,vencimiento_2028,vencimiento_2029,municipio_pdet,municipio_zomac,municipio_pnis,municipio_pnsr_antes_2023,municipio_pnsr_2023,municipio_pnsr_2024
0,AMAZONAS,LETICIA,57,43,12,2,0,43,14,54,0,3,28,19,2,7,1,NO,NO,NO,SI,SI,SI
1,ANTIOQUIA,ABEJORRAL,13,11,1,1,0,11,2,12,0,1,6,2,3,1,1,NO,SI,NO,NO,SI,SI
2,ANTIOQUIA,ALEJANDRÍA,1,0,1,0,0,0,1,0,0,1,1,0,0,0,0,NO,SI,NO,NO,SI,SI
3,ANTIOQUIA,AMAGÁ,22,19,2,1,0,19,3,21,0,1,4,7,0,10,1,NO,NO,NO,NO,NO,NO
4,ANTIOQUIA,AMALFI,15,13,2,0,0,13,2,14,0,1,2,11,2,0,0,SI,SI,SI,SI,SI,SI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
983,VAUPÉS,MITÚ,8,2,2,2,2,2,6,7,0,1,2,5,1,0,0,NO,NO,NO,SI,SI,SI
984,VICHADA,CUMARIBO,4,1,3,0,0,1,3,4,0,0,2,2,0,0,0,NO,SI,SI,SI,SI,SI
985,VICHADA,LA PRIMAVERA,3,3,0,0,0,3,0,3,0,0,1,2,0,0,0,NO,NO,NO,SI,SI,SI
986,VICHADA,PUERTO CARREÑO,24,20,4,0,0,20,4,23,0,1,8,8,2,5,1,NO,NO,NO,SI,SI,SI


## Vista Prestadores + Info poblacional 2025

In [883]:
conn.execute("DROP VIEW IF EXISTS prest_agr_2025")

In [884]:
query = """
CREATE VIEW prest_agr_2025 AS
SELECT 
    pa.*, 
    dpg.*
FROM prest_agr pa
JOIN data_pob_g dpg
  ON pa.depa_nombre = dpg.depa_nombre
 AND pa.muni_nombre = dpg.muni_nombre
WHERE dpg.AÑO = 2025;
"""

conn.execute(query)
conn.commit()

In [885]:
# Leer la vista prest_agr_2025 como DataFrame
prest_agr_2025 = pd.read_sql_query("SELECT * FROM prest_agr_2025", conn)
prest_agr_2025

,depa_nombre,muni_nombre,total_prestadores,profesional_independiente,ips,objeto_social_diferente,transporte_especial,natural,juridico,privada,mixta,publica,vencimiento_2025,vencimiento_2026,vencimiento_2027,vencimiento_2028,vencimiento_2029,municipio_pdet,municipio_zomac,municipio_pnis,municipio_pnsr_antes_2023,municipio_pnsr_2023,municipio_pnsr_2024,MPIO,depa_nombre:1,muni_nombre:1,AÑO,total_pob,total_cabecera,total_cprd,hombres_cabecera,hombres_cprd,hombres_total,mujeres_cabecera,mujeres_cprd,mujeres_total,hombres_primera_infancia,hombres_infancia,hombres_adolescencia,hombres_juventud,hombres_adultez,hombres_vejez,mujeres_primera_infancia,mujeres_infancia,mujeres_adolescencia,mujeres_juventud,mujeres_adultez,mujeres_vejez,etnia_cabecera,etnia_cprd,etnia_total,superficie,region
0,ANTIOQUIA,MEDELLÍN,7364,6549,621,179,15,6549,815,7348,2,14,3487,2355,585,658,279,NO,NO,NO,NO,NO,NO,5001,ANTIOQUIA,MEDELLÍN,2025,2634570,2593542,41028,1219707,20731,1240438,1373835,20297,1394132,82633,93138,99872,189152,588771,186872,79227,89761,97165,185718,657696,284565,69043,293,69336,350.666623,Región Eje Cafetero
1,ANTIOQUIA,ABEJORRAL,13,11,1,1,0,11,2,12,0,1,6,2,3,1,1,NO,SI,NO,NO,SI,SI,5002,ANTIOQUIA,ABEJORRAL,2025,21622,9369,12253,4599,6700,11299,4770,5553,10323,719,925,1006,1322,4919,2408,708,841,854,1074,4459,2387,78,8,86,497.566212,Región Eje Cafetero
2,ANTIOQUIA,ALEJANDRÍA,1,0,1,0,0,0,1,0,0,1,1,0,0,0,0,NO,SI,NO,NO,SI,SI,5021,ANTIOQUIA,ALEJANDRÍA,2025,4989,2960,2029,1380,1087,2467,1580,942,2522,232,231,210,288,1026,480,228,217,185,262,1077,553,15,1,16,119.467683,Región Eje Cafetero
3,ANTIOQUIA,AMAGÁ,22,19,2,1,0,19,3,21,0,1,4,7,0,10,1,NO,NO,NO,NO,NO,NO,5030,ANTIOQUIA,AMAGÁ,2025,32628,17156,15472,8192,7809,16001,8964,7663,16627,1132,1257,1305,2210,7425,2672,1089,1222,1279,2115,7853,3069,163,18,181,89.245903,Región Eje Cafetero
4,ANTIOQUIA,AMALFI,15,13,2,0,0,13,2,14,0,1,2,11,2,0,0,SI,SI,SI,SI,SI,SI,5031,ANTIOQUIA,AMALFI,2025,28059,17098,10961,8046,5939,13985,9052,5022,14074,1223,1455,1465,1942,5856,2044,1178,1413,1429,1898,6126,2030,131,20,151,1301.657433,Región Eje Cafetero
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
917,VAUPÉS,MITÚ,8,2,2,2,2,2,6,7,0,1,2,5,1,0,0,NO,NO,NO,SI,SI,SI,97001,VAUPÉS,MITÚ,2025,35749,10719,25030,5531,13053,18584,5188,11977,17165,2870,2702,2735,3990,5146,1141,2730,2613,2783,3439,4443,1157,6964,23993,30957,16089.802954,Región Llano
918,VICHADA,PUERTO CARREÑO,24,20,4,0,0,20,4,23,0,1,8,8,2,5,1,NO,NO,NO,SI,SI,SI,99001,VICHADA,PUERTO CARREÑO,2025,22963,17605,5358,8827,2829,11656,8778,2529,11307,1187,1121,1239,2218,4855,1036,1140,1098,1155,2018,4920,976,3631,2940,6571,12175.639571,Región Llano
919,VICHADA,LA PRIMAVERA,3,3,0,0,0,3,0,3,0,0,1,2,0,0,0,NO,NO,NO,SI,SI,SI,99524,VICHADA,LA PRIMAVERA,2025,11380,7020,4360,3515,2346,5861,3505,2014,5519,589,635,721,1007,2315,594,536,629,669,926,2246,513,497,1881,2378,18188.762851,Región Llano
920,VICHADA,SANTA ROSALÍA,1,1,0,0,0,1,0,1,0,0,1,0,0,0,0,NO,NO,NO,SI,SI,SI,99624,VICHADA,SANTA ROSALÍA,2025,4732,2935,1797,1438,960,2398,1497,837,2334,211,281,294,378,1029,205,226,293,301,375,905,234,376,881,1257,3930.629399,Región Llano


In [886]:
query_no_match = """
SELECT 
    pa.depa_nombre, 
    pa.muni_nombre
FROM prest_agr pa
LEFT JOIN data_pob_g dpg
  ON pa.depa_nombre = dpg.depa_nombre
 AND pa.muni_nombre = dpg.muni_nombre
WHERE dpg.muni_nombre IS NULL
"""
df_no_match = pd.read_sql_query(query_no_match, conn)
df_no_match

,depa_nombre,muni_nombre
0,ANTIOQUIA,ANZA
1,ANTIOQUIA,DON MATÍAS
2,ANTIOQUIA,ENTRERRIOS
3,ANTIOQUIA,GUATAPE
4,ANTIOQUIA,ITAGUI
...,...,...
61,SAN ANDRÉS Y PROVIDENCIA,SAN ANDRÉS
62,SANTA MARTA,SANTA MARTA
63,SANTANDER,CHIMA
64,SANTANDER,LEBRÍJA


Hacen falta 65 datos, se revisa uno por uno y se realizan las correcciones pertinentes. 

In [887]:
conn.execute("DROP VIEW IF EXISTS prest_agr_2025")

In [ ]:
query = """
CREATE VIEW prest_agr_2025 AS
SELECT 
    pa.*, 
    dpg.*
FROM 
(
    SELECT
        *,
        CASE muni_nombre
            WHEN 'ANZA' THEN 'ANZÁ'
            WHEN 'DON MATÍAS' THEN 'DONMATÍAS'
            WHEN 'ENTRERRIOS' THEN 'ENTRERRÍOS'
            WHEN 'GUATAPE' THEN 'GUATAPÉ'
            WHEN 'ITAGUI' THEN 'ITAGÜÍ'
            WHEN 'SAN ANDRÉS' THEN 'SAN ANDRÉS'
            WHEN 'SAN PEDRO' THEN 'SAN PEDRO'
            WHEN 'SAN PEDRO DE URABA' THEN 'SAN PEDRO DE URABÁ'
            WHEN 'SONSON' THEN 'SONSÓN'
            WHEN 'BARRANQUILLA' THEN 'BARRANQUILLA'
            WHEN 'BOGOTÁ' THEN 'BOGOTÁ, D.C.'
            WHEN 'TURBANA' THEN 'TURBANA'
            WHEN 'GAMEZA' THEN 'GAMEZA'
            WHEN 'GÜICÁN' THEN 'GÜICÁN'
            WHEN 'UMBITA' THEN 'UMBITA'
            WHEN 'BUENAVENTURA' THEN 'BUENAVENTURA'
            WHEN 'CALI' THEN 'CALI'
            WHEN 'BELÉN DE LOS ANDAQUIES' THEN 'BELÉN DE LOS ANDAQUÍES'
            WHEN 'EL PAUJIL' THEN 'EL PAUJIL'
            WHEN 'CARTAGENA' THEN 'CARTAGENA'
            WHEN 'CHAMEZA' THEN 'CHAMEZA'
            WHEN 'PAEZ' THEN 'PÁEZ'
            WHEN 'PIENDAMÓ' THEN 'PIENDAMÓ'
            WHEN 'TORIBIO' THEN 'TORIBÍO'
            WHEN 'MANAURE' THEN 'MANAURE'
            WHEN 'ALTO BAUDO' THEN 'ALTO BAUDÓ'
            WHEN 'BELÉN DE BAJIRÁ' THEN 'BELÉN DE BAJIRÁ'
            WHEN 'BOJAYA' THEN 'BOJAYÁ'
            WHEN 'CARMEN DEL DARIEN' THEN 'CARMEN DEL DARIÉN'
            WHEN 'CAQUEZA' THEN 'CAQUEZA'
            WHEN 'FOMEQUE' THEN 'FÓMEQUE'
            WHEN 'MACHETA' THEN 'MACHETA'
            WHEN 'SAN JUAN DE RÍO SECO' THEN 'SAN JUAN DE RÍO SECO'
            WHEN 'VILLA DE SAN DIEGO DE UBATE' THEN 'VILLA DE SAN DIEGO DE ÚBATÉ'
            WHEN 'PURÍSIMA' THEN 'PURÍSIMA'
            WHEN 'IQUIRA' THEN 'IQUIRA'
            WHEN 'CERRO SAN ANTONIO' THEN 'CERRO SAN ANTONIO'
            WHEN 'EL PIÑON' THEN 'EL PIÑÓN'
            WHEN 'SABANAS DE SAN ANGEL' THEN 'SABANAS DE SAN ÁNGEL'
            WHEN 'ANCUYÁ' THEN 'ANCUYÁ'
            WHEN 'CONSACA' THEN 'CONSACÁ'
            WHEN 'CUASPUD' THEN 'CUASPÚD'
            WHEN 'MAGÜI' THEN 'MAGÜÍ'
            WHEN 'TUMACO' THEN 'TUMACO'
            WHEN 'ABREGO' THEN 'ÁBREGO'
            WHEN 'CACHIRÁ' THEN 'CACHIRÁ'
            WHEN 'CÚCUTA' THEN 'CÚCUTA'
            WHEN 'LEGUÍZAMO' THEN 'LEGUÍZAMO'
            WHEN 'ARMENIA' THEN 'ARMENIA'
            WHEN 'BUENAVISTA' THEN 'BUENAVISTA'
            WHEN 'CALARCA' THEN 'CALARCÁ'
            WHEN 'CIRCASIA' THEN 'CIRCASIA'
            WHEN 'CÓRDOBA' THEN 'CÓRDOBA'
            WHEN 'FILANDIA' THEN 'FILANDIA'
            WHEN 'GÉNOVA' THEN 'GÉNOVA'
            WHEN 'LA TEBAIDA' THEN 'LA TEBAIDA'
            WHEN 'MONTENEGRO' THEN 'MONTENEGRO'
            WHEN 'PIJAO' THEN 'PIJAO'
            WHEN 'QUIMBAYA' THEN 'QUIMBAYA'
            WHEN 'SALENTO' THEN 'SALENTO'
            WHEN 'PROVIDENCIA' THEN 'SAN ANDRÉS'
            WHEN 'SANTA MARTA' THEN 'SANTA MARTA'
            WHEN 'CHIMA' THEN 'CHIMA'
            WHEN 'LEBRÍJA' THEN 'LEBRIJA'
            WHEN 'SINCÉ' THEN 'SINCELEJO'
            ELSE muni_nombre
        END AS muni_nombre_corr,

        CASE depa_nombre
            WHEN 'ANTIOQUIA' THEN 'ANTIOQUIA'
            WHEN 'BARRANQUILLA' THEN 'ATLÁNTICO'
            WHEN 'BOGOTÁ DC' THEN 'BOGOTÁ, D.C.'
            WHEN 'BOLÍVAR' THEN 'BOLÍVAR'
            WHEN 'BOYACÁ' THEN 'BOYACÁ'
            WHEN 'BUENAVENTURA' THEN 'VALLE DEL CAUCA'
            WHEN 'CALI' THEN 'VALLE DEL CAUCA'
            WHEN 'CAQUETÁ' THEN 'CAQUETÁ'
            WHEN 'CARTAGENA' THEN 'BOLÍVAR'
            WHEN 'CASANARE' THEN 'CASANARE'
            WHEN 'CAUCA' THEN 'CAUCA'
            WHEN 'CESAR' THEN 'CESAR'
            WHEN 'CHOCÓ' THEN 'CHOCÓ'
            WHEN 'CUNDINAMARCA' THEN 'CUNDINAMARCA'
            WHEN 'CÓRDOBA' THEN 'CÓRDOBA'
            WHEN 'HUILA' THEN 'HUILA'
            WHEN 'MAGDALENA' THEN 'MAGDALENA'
            WHEN 'NARIÑO' THEN 'NARIÑO'
            WHEN 'NORTE DE SANTANDER' THEN 'NORTE DE SANTANDER'
            WHEN 'PUTUMAYO' THEN 'PUTUMAYO'
            WHEN 'QUINDÍO' THEN 'QUINDÍO'
            WHEN 'SAN ANDRÉS Y PROVIDENCIA' THEN 'ARCHIPIÉLAGO DE SAN ANDRÉS'
            WHEN 'SANTA MARTA' THEN 'MAGDALENA'
            WHEN 'SANTANDER' THEN 'SANTANDER'
            WHEN 'SUCRE' THEN 'SUCRE'
            ELSE depa_nombre
        END AS depa_nombre_corr
    FROM prest_agr
) pa
JOIN data_pob_g dpg
    ON pa.depa_nombre_corr = dpg.depa_nombre
   AND pa.muni_nombre_corr = dpg.muni_nombre
WHERE dpg.AÑO = 2025;

"""
conn.execute(query)
conn.commit()


In [889]:
# Leer la vista prest_agr_2025 como DataFrame
prest_agr_2025 = pd.read_sql_query("SELECT * FROM prest_agr_2025", conn)
prest_agr_2025

,depa_nombre,muni_nombre,total_prestadores,profesional_independiente,ips,objeto_social_diferente,transporte_especial,natural,juridico,privada,mixta,publica,vencimiento_2025,vencimiento_2026,vencimiento_2027,vencimiento_2028,vencimiento_2029,municipio_pdet,municipio_zomac,municipio_pnis,municipio_pnsr_antes_2023,municipio_pnsr_2023,municipio_pnsr_2024,muni_nombre_corr,depa_nombre_corr,MPIO,depa_nombre:1,muni_nombre:1,AÑO,total_pob,total_cabecera,total_cprd,hombres_cabecera,hombres_cprd,hombres_total,mujeres_cabecera,mujeres_cprd,mujeres_total,hombres_primera_infancia,hombres_infancia,hombres_adolescencia,hombres_juventud,hombres_adultez,hombres_vejez,mujeres_primera_infancia,mujeres_infancia,mujeres_adolescencia,mujeres_juventud,mujeres_adultez,mujeres_vejez,etnia_cabecera,etnia_cprd,etnia_total,superficie,region
0,AMAZONAS,LETICIA,57,43,12,2,0,43,14,54,0,3,28,19,2,7,1,NO,NO,NO,SI,SI,SI,LETICIA,AMAZONAS,91001,AMAZONAS,LETICIA,2025,55691,40511,15180,19934,7928,27862,20577,7252,27829,3466,3439,3486,5006,10486,1979,3309,3361,3439,4658,10827,2235,11973,13737,25710,5717.713561,Región Centro Sur
1,ANTIOQUIA,ABEJORRAL,13,11,1,1,0,11,2,12,0,1,6,2,3,1,1,NO,SI,NO,NO,SI,SI,ABEJORRAL,ANTIOQUIA,5002,ANTIOQUIA,ABEJORRAL,2025,21622,9369,12253,4599,6700,11299,4770,5553,10323,719,925,1006,1322,4919,2408,708,841,854,1074,4459,2387,78,8,86,497.566212,Región Eje Cafetero
2,ANTIOQUIA,ALEJANDRÍA,1,0,1,0,0,0,1,0,0,1,1,0,0,0,0,NO,SI,NO,NO,SI,SI,ALEJANDRÍA,ANTIOQUIA,5021,ANTIOQUIA,ALEJANDRÍA,2025,4989,2960,2029,1380,1087,2467,1580,942,2522,232,231,210,288,1026,480,228,217,185,262,1077,553,15,1,16,119.467683,Región Eje Cafetero
3,ANTIOQUIA,AMAGÁ,22,19,2,1,0,19,3,21,0,1,4,7,0,10,1,NO,NO,NO,NO,NO,NO,AMAGÁ,ANTIOQUIA,5030,ANTIOQUIA,AMAGÁ,2025,32628,17156,15472,8192,7809,16001,8964,7663,16627,1132,1257,1305,2210,7425,2672,1089,1222,1279,2115,7853,3069,163,18,181,89.245903,Región Eje Cafetero
4,ANTIOQUIA,AMALFI,15,13,2,0,0,13,2,14,0,1,2,11,2,0,0,SI,SI,SI,SI,SI,SI,AMALFI,ANTIOQUIA,5031,ANTIOQUIA,AMALFI,2025,28059,17098,10961,8046,5939,13985,9052,5022,14074,1223,1455,1465,1942,5856,2044,1178,1413,1429,1898,6126,2030,131,20,151,1301.657433,Región Eje Cafetero
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
944,VAUPÉS,MITÚ,8,2,2,2,2,2,6,7,0,1,2,5,1,0,0,NO,NO,NO,SI,SI,SI,MITÚ,VAUPÉS,97001,VAUPÉS,MITÚ,2025,35749,10719,25030,5531,13053,18584,5188,11977,17165,2870,2702,2735,3990,5146,1141,2730,2613,2783,3439,4443,1157,6964,23993,30957,16089.802954,Región Llano
945,VICHADA,CUMARIBO,4,1,3,0,0,1,3,4,0,0,2,2,0,0,0,NO,SI,SI,SI,SI,SI,CUMARIBO,VICHADA,99773,VICHADA,CUMARIBO,2025,88392,3102,85290,1544,44614,46158,1558,40676,42234,6452,6390,6395,8475,15202,3244,6285,6171,6101,7667,13461,2549,1273,77907,79180,64509.421614,Región Llano
946,VICHADA,LA PRIMAVERA,3,3,0,0,0,3,0,3,0,0,1,2,0,0,0,NO,NO,NO,SI,SI,SI,LA PRIMAVERA,VICHADA,99524,VICHADA,LA PRIMAVERA,2025,11380,7020,4360,3515,2346,5861,3505,2014,5519,589,635,721,1007,2315,594,536,629,669,926,2246,513,497,1881,2378,18188.762851,Región Llano
947,VICHADA,PUERTO CARREÑO,24,20,4,0,0,20,4,23,0,1,8,8,2,5,1,NO,NO,NO,SI,SI,SI,PUERTO CARREÑO,VICHADA,99001,VICHADA,PUERTO CARREÑO,2025,22963,17605,5358,8827,2829,11656,8778,2529,11307,1187,1121,1239,2218,4855,1036,1140,1098,1155,2018,4920,976,3631,2940,6571,12175.639571,Región Llano


# Exportación de datos agregados para análisis

In [890]:
#Se exportan estos datos para análisis
prest_agr_2025.to_csv("Bases de datos agregada/prest_agr_2025.csv", index=False, encoding='utf-8')